## Import libraries

In [2]:
import sys
sys.path.append('/Users/zoltan/Documents/Rivabar/rivabar')

In [3]:
## Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
import pyproj
from rasterio.crs import CRS
import cartopy.crs as ccrs
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.ticker as mticker
from pystac_client import Client
import rivabar as rb

%matplotlib qt
plt.rcParams['svg.fonttype'] = 'none'

In [53]:
# Remove all rivabar modules from cache
modules_to_remove = [key for key in sys.modules.keys() if key.startswith('rivabar')]
for module in modules_to_remove:
    del sys.modules[module]

# Fresh import
import rivabar as rb

## Branco River, Brazil

You can find the links to the Landsat bands that are needed by running this cell. Clicking the links will download the raster files.

In [4]:
catalog = Client.open("https://landsatlook.usgs.gov/stac-server")
scene_id = "LC82320602014050LGN02"
search = catalog.search(
    collections=["landsat-c2l2-sr"],
    query={"landsat:scene_id": {"eq": scene_id}},
    limit=1
)
item = next(search.items())
for band in ['red', 'green', 'blue', 'swir16']:
    print(item.assets[band].href)

https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2014/232/060/LC08_L2SP_232060_20140219_20200911_02_T1/LC08_L2SP_232060_20140219_20200911_02_T1_SR_B4.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2014/232/060/LC08_L2SP_232060_20140219_20200911_02_T1/LC08_L2SP_232060_20140219_20200911_02_T1_SR_B3.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2014/232/060/LC08_L2SP_232060_20140219_20200911_02_T1/LC08_L2SP_232060_20140219_20200911_02_T1_SR_B2.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2014/232/060/LC08_L2SP_232060_20140219_20200911_02_T1/LC08_L2SP_232060_20140219_20200911_02_T1_SR_B6.TIF


### Create water index image and pick start- and end points for channel belt

In [7]:
cd ..

/Users/zoltan/Documents/Rivabar/rivabar


In [9]:
# Create river without start/end points initially
river = rb.River(
    fname='LC08_L2SP_232060_20140219_20200911_02_T1_SR',
    dirname='data/Branco/',
    file_type='multiple_tifs'
)

# Create and visualize MNDWI first
mndwi = river.create_mndwi(mndwi_threshold=0.01)
river.plot_mndwi();

Creating MNDWI for: LC08_L2SP_232060_20140219_20200911_02_T1_SR


100%|██████████| 4/4 [00:00<00:00,  4.93it/s]


✓ MNDWI created: (7731, 7581)
  - Water pixels: 213,071.0
  - Total pixels: 58,608,711
  - Water percentage: 0.4%


In [10]:
# Pick start and end points
river.get_start_end_points_interactive()

Click two points: first for START, second for END
Start point: (676683.2, 100454.6)
End point: (627360.4, -92694.9)


(np.float64(676683.2142857143),
 np.float64(100454.61038961037),
 np.float64(627360.4383116884),
 np.float64(-92694.88636363635))

### Extract channel centerlines and banklines

In [11]:
river.map_river_banks(
    ch_belt_smooth_factor=1e8,
    ch_belt_half_width=2000,
    remove_smaller_components=False,
    small_hole_threshold=64,
    solidity_filter=False,
    plot_D_primal=True, 
    flip_outlier_edges=True,
    check_edges=False
)

Processing river: LC08_L2SP_232060_20140219_20200911_02_T1_SR
Start point: (676683.2142857143, 100454.61038961037)
End point: (627360.4383116884, -92694.88636363635)
Phase 1: Initial setup and skeletonization


100%|██████████| 4/4 [00:00<00:00,  4.89it/s]


running skeletonization
Phase 2: Finding main path with fallbacks
Phase 3: Processing graph and extending edges


100%|██████████| 208/208 [00:00<00:00, 212.05it/s]


Phase 4: Extracting and smoothing main path
Phase 5: Polygonizing centerline network


...

100%|██████████| 189/189 [00:01<00:00, 128.28it/s]


Phase 12: Get bank coordinates for main channel


100%|██████████| 189/189 [00:00<00:00, 2399.70it/s]

✓ Successfully processed LC08_L2SP_232060_20140219_20200911_02_T1_SR
  - Directed graph: 126 nodes, 189 edges
  - Rook graph: 65 polygons
  - Primal graph: 126 nodes


True

### Plot width data for main channel

In [12]:
s, w = river.get_channel_widths()
plt.figure(figsize=(12, 4))
plt.plot(s, w)
plt.xlabel('along-channel distance (m)')
plt.ylabel('channel width (m)');

### Plot banks and centerline for main channel

In [13]:
lbx = river.main_channel_banks['left_bank'].xy[0]
lby = river.main_channel_banks['left_bank'].xy[1]
rbx = river.main_channel_banks['right_bank'].xy[0]
rby = river.main_channel_banks['right_bank'].xy[1]
plt.figure()
plt.plot(lbx, lby, 'k')
plt.plot(rbx, rby, 'k')
plt.axis('equal');

### Save results to shapefiles

In [14]:
rb.write_shapefiles_and_graphs(river._G_rook, river._D_primal, 
                               river._dataset, river.dirname, 'Branco')

### Save results to pickle file

In [233]:
river.save_results('Branco_results.pkl')

Saved results to Branco_results.pkl


In [15]:
from rivabar.river import River
river = River.load_results('Branco_results.pkl')

Loaded results from Branco_results.pkl


## Mamore River, Bolivia


Data download links:

In [16]:
catalog = Client.open("https://landsatlook.usgs.gov/stac-server")
scene_id = "LC82320712016168LGN01"
search = catalog.search(
    collections=["landsat-c2l2-sr"],
    query={"landsat:scene_id": {"eq": scene_id}},
    limit=1
)
item = next(search.items())
for band in ['red', 'green', 'blue', 'swir16']:
    print(item.assets[band].href)

https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2016/232/071/LC08_L2SP_232071_20160616_20200906_02_T1/LC08_L2SP_232071_20160616_20200906_02_T1_SR_B4.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2016/232/071/LC08_L2SP_232071_20160616_20200906_02_T1/LC08_L2SP_232071_20160616_20200906_02_T1_SR_B3.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2016/232/071/LC08_L2SP_232071_20160616_20200906_02_T1/LC08_L2SP_232071_20160616_20200906_02_T1_SR_B2.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2016/232/071/LC08_L2SP_232071_20160616_20200906_02_T1/LC08_L2SP_232071_20160616_20200906_02_T1_SR_B6.TIF


In [17]:
# in this case, the start and end points are known and are provided as arguments
river = rb.River(
    fname='LC08_L2SP_232071_20160616_20200906_02_T1_SR',
    dirname='data/Mamore/',
    start_x = 303810,
    start_y = -1851620,
    end_x = 285603,
    end_y = -1666131,
    file_type='multiple_tifs'
)

In [18]:
river.map_river_banks(
    ch_belt_smooth_factor=1e8,
    ch_belt_half_width=2000,
    remove_smaller_components=False,
    small_hole_threshold=64,
    solidity_filter=False,
    plot_D_primal=True, 
    flip_outlier_edges=False,
    check_edges=False
)

Processing river: LC08_L2SP_232071_20160616_20200906_02_T1_SR
Start point: (303810, -1851620)
End point: (285603, -1666131)
Phase 1: Initial setup and skeletonization


100%|██████████| 4/4 [00:00<00:00,  4.51it/s]


running skeletonization
Phase 2: Finding main path with fallbacks
no path between start_ind and end_ind
Phase 3: Processing graph and extending edges


100%|██████████| 66/66 [00:00<00:00, 610.29it/s]


Phase 4: Extracting and smoothing main path
Phase 5: Polygonizing centerline network


...

100%|██████████| 37/37 [00:00<00:00, 64.11it/s]


Phase 12: Get bank coordinates for main channel


100%|██████████| 37/37 [00:00<00:00, 743.52it/s]

✓ Successfully processed LC08_L2SP_232071_20160616_20200906_02_T1_SR
  - Directed graph: 26 nodes, 37 edges
  - Rook graph: 14 polygons
  - Primal graph: 26 nodes


True

### Plot width data for main channel

In [22]:
s, w = river.get_channel_widths()
plt.figure(figsize=(12, 4))
plt.plot(s, w) 
plt.xlabel('along-channel distance (m)')
plt.ylabel('channel width (m)');

### Plot meander wavelength and channel width data

In [23]:
fig, ax = river.plot_overview()
df, curv, s, loc_zero_curv, xsmooth, ysmooth = river.analyze_wavelength_and_width(ax=ax, delta_s=5, smoothing_factor=0.5*1e7, min_sinuosity=1.3, dx=30)
df.head()

100%|██████████| 37/37 [00:00<00:00, 2458.44it/s]


,wavelengths (m),sinuosities,mean widths (m),std. dev. of widths (m),along-channel distance (km)
0,2513.154125,2.034175,144.632450,28.204387,8.288075
1,6504.541947,1.331073,219.812418,58.839936,12.489620
2,5479.132524,1.655118,204.246520,44.184479,16.935367
3,3809.984967,1.323008,221.357291,51.070163,20.467039
4,3298.801311,3.181899,197.604339,29.580510,25.693113


### Map a longer river segment

In [24]:
river = rb.River(
    fname='L8_mndwi_Mamore_2015_UTM.tif',
    dirname='/Users/zoltan/Documents/Channels/Fluvial/Mamore/',
    start_x = 303559.07,
    start_y = 8148946.7,
    end_x = 271009.56,
    end_y = 8549337.76,
    file_type='water_index'
)

river.map_river_banks(
    ch_belt_smooth_factor=1e8,
    ch_belt_half_width=2000,
    remove_smaller_components=False,
    small_hole_threshold=64,
    solidity_filter=False,
    plot_D_primal=True,
    flip_outlier_edges=False,
    check_edges=False
)


Processing river: L8_mndwi_Mamore_2015_UTM.tif
Start point: (303559.07, 8148946.7)
End point: (271009.56, 8549337.76)
Phase 1: Initial setup and skeletonization
running skeletonization
Phase 2: Finding main path with fallbacks
Phase 3: Processing graph and extending edges


100%|██████████| 230/230 [00:00<00:00, 344.17it/s]


Phase 4: Extracting and smoothing main path
Phase 5: Polygonizing centerline network


100%|██████████| 257/257 [00:00<00:00, 695.35it/s] 


Phase 6: Creating channel belt and boundaries
Phase 7: Create UTM geodataframe


...

100%|██████████| 28/28 [00:00<00:00, 31.22it/s]


Phase 12: Get bank coordinates for main channel


100%|██████████| 28/28 [00:00<00:00, 156.67it/s]

✓ Successfully processed L8_mndwi_Mamore_2015_UTM.tif
  - Directed graph: 20 nodes, 28 edges
  - Rook graph: 11 polygons
  - Primal graph: 20 nodes


True

In [25]:
river.summary()


=== River Summary: L8_mndwi_Mamore_2015_UTM.tif ===
Processed: True
Successful: True
Start point: (303559.1, 8148946.7)
End point: (271009.6, 8549337.8)
Directed graph: 20 nodes, 28 edges
Rook graph: 11 polygons
Primal graph: 20 nodes
MNDWI shape: (13843, 2926)
Main path: 18 edges
Channel width: 346.7 ± 124.7 m
Channel length: 929662.2 m


## Brahmaputra River, India / Bangladesh


Data download links:

In [26]:
catalog = Client.open("https://landsatlook.usgs.gov/stac-server")
scene_id = "LC81380422019014LGN00"
search = catalog.search(
    collections=["landsat-c2l2-sr"],
    query={"landsat:scene_id": {"eq": scene_id}},
    limit=1
)
item = next(search.items())
for band in ['red', 'green', 'blue', 'swir16']:
    print(item.assets[band].href)

https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2019/138/042/LC08_L2SP_138042_20190114_20200830_02_T1/LC08_L2SP_138042_20190114_20200830_02_T1_SR_B4.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2019/138/042/LC08_L2SP_138042_20190114_20200830_02_T1/LC08_L2SP_138042_20190114_20200830_02_T1_SR_B3.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2019/138/042/LC08_L2SP_138042_20190114_20200830_02_T1/LC08_L2SP_138042_20190114_20200830_02_T1_SR_B2.TIF
https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2019/138/042/LC08_L2SP_138042_20190114_20200830_02_T1/LC08_L2SP_138042_20190114_20200830_02_T1_SR_B6.TIF


In [27]:
river = rb.River(
    fname='LC08_L2SP_138042_20190114_20200830_02_T1_SR',
    dirname='data/Brahmaputra/',
    start_x = 859250.5,
    start_y = 2903119.2,
    end_x = 767358.1,
    end_y = 2776067.9,
    file_type='multiple_tifs'
)

river.map_river_banks(
    ch_belt_smooth_factor=1e8,
    ch_belt_half_width=2000,
    remove_smaller_components=True,
    small_hole_threshold=64,
    solidity_filter=False,
    plot_D_primal=True,
    flip_outlier_edges=True,
    check_edges=True
)


Processing river: LC08_L2SP_138042_20190114_20200830_02_T1_SR
Start point: (859250.5, 2903119.2)
End point: (767358.1, 2776067.9)
Phase 1: Initial setup and skeletonization


100%|██████████| 4/4 [00:00<00:00,  4.47it/s]


running skeletonization
Phase 2: Finding main path with fallbacks
Phase 3: Processing graph and extending edges


100%|██████████| 213/213 [00:04<00:00, 46.50it/s]


Phase 4: Extracting and smoothing main path
Phase 5: Polygonizing centerline network


...

100%|██████████| 802/802 [00:06<00:00, 132.44it/s]


Found 2 source nodes: [5, 38]
Found 3 sink nodes: [3, 163, 313]
After corrections, found 2 source nodes: [5, 51]
After corrections, found 1 sink nodes: [3]
Phase 12: Get bank coordinates for main channel


100%|██████████| 802/802 [00:00<00:00, 2981.92it/s]

✓ Successfully processed LC08_L2SP_138042_20190114_20200830_02_T1_SR
  - Directed graph: 525 nodes, 802 edges
  - Rook graph: 274 polygons
  - Primal graph: 525 nodes


True

In [28]:
river.summary()


=== River Summary: LC08_L2SP_138042_20190114_20200830_02_T1_SR ===
Processed: True
Successful: True
Start point: (859250.5, 2903119.2)
End point: (767358.1, 2776067.9)
Directed graph: 525 nodes, 802 edges
Rook graph: 274 polygons
Primal graph: 525 nodes
MNDWI shape: (7691, 7541)
Main path: 122 edges
Channel width: 746.2 ± 282.1 m
Channel length: 221232.1 m


### Exceedance probability plot of island areas

In [29]:
island_areas = []
channel_areas = []
total_areas = []
degrees = []
lengths = []
for node in river._G_rook:
    poly1 = river._G_rook.nodes()[node]['bank_polygon']
    poly2 = river._G_rook.nodes()[node]['cl_polygon']
    if poly1.area > 0 and poly2.area > poly1.area:
        island_areas.append(poly1.area)
        channel_areas.append(poly2.area - poly1.area)
        total_areas.append(poly2.area)
        degrees.append(river._G_rook.degree(node))
        lengths.append(poly1.length)
        
island_areas_sorted = np.sort(island_areas)
island_areas_sorted = island_areas_sorted[:-2]
exceedance = 1.-np.arange(1.,len(island_areas_sorted) + 1.)/len(island_areas_sorted)

plt.figure()
plt.loglog(island_areas_sorted, exceedance)
plt.xlabel('island area (m2)')
plt.ylabel('exceedance probability');

### Histogram of node degree distribution

In [30]:
plt.figure()
n, bins, patches = plt.hist(degrees, bins=np.arange(1.5, 25.6, 1))

plt.bar(np.arange(2,26), n)
plt.xticks(np.arange(2,26))
plt.xlabel('node degree')
plt.ylabel('count')
plt.title('number of neighbors in island neighborhood graph');

## Purus River, Brazil

The water mask was generated in Google Earth Engine.

Initial output from GEE needs to be converted to UTM coordinates before running rivabar.

Link to data files: https://utexas.box.com/s/cb6wsihdoykbti1iju57rs2cfrszqmyx

In [31]:
river = rb.River(
    dirname = "data/Purus/",
    fname = "L8_mndwi_Purus_2017_UTM.tif",
    start_x = 363166.2,
    start_y = -1011937.1,
    end_x = 1254178.3,
    end_y = -528280.7,
    file_type='water_index'
)

river.map_river_banks(
    ch_belt_smooth_factor=1e8,
    ch_belt_half_width=2000,
    remove_smaller_components=True,
    small_hole_threshold=64,
    solidity_filter=False,
    plot_D_primal=True,
    flip_outlier_edges=False,
    check_edges=False
)

Processing river: L8_mndwi_Purus_2017_UTM.tif
Start point: (363166.2, -1011937.1)
End point: (1254178.3, -528280.7)
Phase 1: Initial setup and skeletonization
running skeletonization
Phase 2: Finding main path with fallbacks
Phase 3: Processing graph and extending edges


100%|██████████| 282/282 [00:02<00:00, 104.95it/s]


Phase 4: Extracting and smoothing main path
Phase 5: Polygonizing centerline network


100%|██████████| 702/702 [00:02<00:00, 240.78it/s] 


Phase 6: Creating channel belt and boundaries
Phase 7: Create UTM geodataframe


...

100%|██████████| 37/37 [00:03<00:00, 10.29it/s]


Phase 12: Get bank coordinates for main channel


100%|██████████| 37/37 [00:00<00:00, 107.96it/s]

✓ Successfully processed L8_mndwi_Purus_2017_UTM.tif
  - Directed graph: 26 nodes, 37 edges
  - Rook graph: 14 polygons
  - Primal graph: 26 nodes


True

In [32]:
river.summary()


=== River Summary: L8_mndwi_Purus_2017_UTM.tif ===
Processed: True
Successful: True
Start point: (363166.2, -1011937.1)
End point: (1254178.3, -528280.7)
Directed graph: 26 nodes, 37 edges
Rook graph: 14 polygons
Primal graph: 26 nodes
MNDWI shape: (17232, 29991)
Main path: 17 edges
Channel width: 350.5 ± 188.8 m
Channel length: 2802522.0 m


### Plot width data for main channel

In [33]:
s, w = river.get_channel_widths()
plt.figure(figsize=(12, 4))
plt.plot(s, w) 
plt.xlabel('along-channel distance (m)')
plt.ylabel('channel width (m)');

### Plot meander wavelength and channel width data

In [ ]:
fig, ax = river.plot_overview()
df, curv, s, loc_zero_curv, xsmooth, ysmooth = river.analyze_wavelength_and_width(ax=ax, delta_s=60.0, smoothing_factor=0.5*1e8, min_sinuosity=1.1, dx=30)
df.head()

100%|██████████| 37/37 [00:00<00:00, 666.45it/s]


,wavelengths (m),sinuosities,mean widths (m),std. dev. of widths (m),along-channel distance (km)
0,1635.420770,1.222824,125.088781,30.060030,2.773843
1,3825.558024,3.087901,141.049716,27.073181,6.243596
2,3743.019083,1.720969,135.290306,27.660051,10.771980
3,3585.896290,1.366456,147.289445,32.129947,13.592080
4,3114.236979,1.156250,150.689963,40.897407,15.780499


### Merge two files into a single TIF file (often needed when downloading files from GEE)

In [ ]:
dirname = "../data/Purus/"
fname1 = 'L8_mndwi_mosaic_Purus_2019_2020-0000000000-0000000000_UTM.tif'
fname2 = 'L8_mndwi_mosaic_Purus_2019_2020-0000000000-0000032768_UTM.tif'

from rasterio.merge import merge

# Open the first raster dataset
src1 = rasterio.open(dirname+fname1) 
meta1 = src1.meta
# Open the second raster dataset
src2 = rasterio.open(dirname+fname2)
# Merge the two arrays
merged_arr, merged_transform = merge([src1, src2])
merged_arr = merged_arr[0, :, :]
# Update the metadata with the merged shape and transform
meta1.update({
    'height': merged_arr.shape[0],
    'width': merged_arr.shape[1],
    'transform': merged_transform
})
# Write the merged raster dataset to disk
with rasterio.open(dirname + 'L8_mndwi_mosaic_Purus_2019_2020_UTM.tif', 'w', **meta1) as dst:
    dst.write(merged_arr, 1)

## Mississippi River, USA

In [35]:
# Create river without start/end points initially
river = rb.River(
    dirname = "/Users/zoltan/Documents/Channels/Fluvial/Mississippi/",
    fname = "L8_mndwi_Mississippi_2019_UTM_edited_2.tif",
    file_type='water_index'
)

# Create and visualize MNDWI first
mndwi = river.create_mndwi(mndwi_threshold=0.5)
river.plot_mndwi();

Creating MNDWI for: L8_mndwi_Mississippi_2019_UTM_edited_2.tif
✓ MNDWI created: (30749, 9500)
  - Water pixels: 4,717,950.0
  - Total pixels: 292,115,500
  - Water percentage: 1.6%


In [36]:
river = rb.River(
    dirname = "/Users/zoltan/Documents/Channels/Fluvial/Mississippi/",
    fname = "L8_mndwi_Mississippi_2019_UTM_edited_2.tif",
    start_x = 845257.5,
    start_y = 4099939.1,
    end_x = 877774.5,
    end_y = 3212403.1,
    file_type='water_index'
)

river.map_river_banks(
    ch_belt_smooth_factor=1e8,
    ch_belt_half_width=2000,
    mndwi_threshold=0.5,
    remove_smaller_components=True,
    small_hole_threshold=100,
    solidity_filter=False,
    plot_D_primal=True,
    flip_outlier_edges=False,
    check_edges=False
)


Processing river: L8_mndwi_Mississippi_2019_UTM_edited_2.tif
Start point: (845257.5, 4099939.1)
End point: (877774.5, 3212403.1)
Phase 1: Initial setup and skeletonization
running skeletonization
Phase 2: Finding main path with fallbacks
Phase 3: Processing graph and extending edges


100%|██████████| 1362/1362 [00:56<00:00, 24.20it/s]


Phase 4: Extracting and smoothing main path
Phase 5: Polygonizing centerline network


100%|██████████| 5553/5553 [02:50<00:00, 32.60it/s] 


Phase 6: Creating channel belt and boundaries
Phase 7: Create UTM geodataframe


...

100%|██████████| 1427/1427 [01:16<00:00, 18.61it/s]


Phase 12: Get bank coordinates for main channel


100%|██████████| 1427/1427 [00:00<00:00, 2146.33it/s]

✓ Successfully processed L8_mndwi_Mississippi_2019_UTM_edited_2.tif
  - Directed graph: 939 nodes, 1427 edges
  - Rook graph: 483 polygons
  - Primal graph: 939 nodes


True

In [37]:
river.summary()


=== River Summary: L8_mndwi_Mississippi_2019_UTM_edited_2.tif ===
Processed: True
Successful: True
Start point: (845257.5, 4099939.1)
End point: (877774.5, 3212403.1)
Directed graph: 939 nodes, 1427 edges
Rook graph: 483 polygons
Primal graph: 939 nodes
MNDWI shape: (30749, 9500)
Main path: 360 edges
Channel width: 1244.1 ± 971.4 m
Channel length: 1633798.8 m


In [38]:
island_areas = []
channel_areas = []
total_areas = []
degrees = []
lengths = []
for node in river._G_rook:
    poly1 = river._G_rook.nodes()[node]['bank_polygon']
    poly2 = river._G_rook.nodes()[node]['cl_polygon']
    if poly1.area > 0 and poly2.area > poly1.area:
        island_areas.append(poly1.area)
        channel_areas.append(poly2.area - poly1.area)
        total_areas.append(poly2.area)
        degrees.append(river._G_rook.degree(node))
        lengths.append(poly1.length)
        
island_areas_sorted = np.sort(island_areas)
island_areas_sorted = island_areas_sorted[:-2]
exceedance = 1.-np.arange(1.,len(island_areas_sorted) + 1.)/len(island_areas_sorted)

plt.figure()
plt.loglog(island_areas_sorted, exceedance)
plt.xlabel('island area (m2)')
plt.ylabel('exceedance probability');

In [39]:
degrees_sorted = np.sort(degrees)
degrees_sorted = degrees_sorted[:-2]
exceedance = 1.-np.arange(1.,len(degrees_sorted) + 1.)/len(degrees_sorted)

plt.figure()
plt.semilogy(degrees_sorted, exceedance)
plt.xlabel('degrees')
plt.ylabel('exceedance probability');

In [40]:
plt.figure()
n, bins, patches = plt.hist(degrees, bins=np.arange(1.5, 25.6, 1))

plt.bar(np.arange(2,26), n)
plt.xticks(np.arange(2,26))
plt.xlabel('node degree')
plt.ylabel('count')
plt.title('number of neighbors in island neighborhood graph')
plt.xlim(1.5, 25.5);

In [41]:
fig, ax = river.plot_overview()
df, curv, s, loc_zero_curv, xsmooth, ysmooth = river.analyze_wavelength_and_width(ax=ax, 
                        delta_s=30.0, smoothing_factor=2*1e8, min_sinuosity=1.1, dx=30)
df.head()

100%|██████████| 1427/1427 [00:00<00:00, 5386.24it/s]


,wavelengths (m),sinuosities,mean widths (m),std. dev. of widths (m),along-channel distance (km)
0,10222.493079,1.132705,1068.426441,167.413821,2.890764
1,15867.950157,1.358282,1271.141648,225.926119,50.600527
2,11333.194871,1.213759,948.234017,121.457520,59.401654
3,18356.293210,1.507196,1431.910746,154.125922,79.076168
4,17625.346518,1.345758,1172.606071,175.797833,95.290684


## Adelaide River, Australia

Water mask was generated from Planet Labs data.

Link to data file: https://utexas.box.com/s/g5mgrpw3pir8lwfnlx7nnexggg7o59an

In [54]:
river = rb.River(
    dirname = "data/Adelaide/",
    fname = "ndwi_mosaic.tif",
    start_x = 753443.4,
    start_y = 8598434.2,
    end_x = 743089.7,
    end_y = 8648110.2,
    file_type='water_index'
)

river.map_river_banks(
    ch_belt_smooth_factor=1e8,
    ch_belt_half_width=2000,
    mndwi_threshold=0.5,
    remove_smaller_components=False,
    small_hole_threshold=64,
    solidity_filter=False,
    plot_D_primal=True,
    flip_outlier_edges=False,
    check_edges=False
)

Processing river: ndwi_mosaic.tif
Start point: (753443.4, 8598434.2)
End point: (743089.7, 8648110.2)
Phase 1: Initial setup and skeletonization
running skeletonization
Phase 2: Finding main path with fallbacks
Phase 3: Processing graph and extending edges


100%|██████████| 62/62 [00:00<00:00, 267.72it/s]


Phase 4: Extracting and smoothing main path
Phase 5: Polygonizing centerline network


100%|██████████| 80/80 [00:00<00:00, 1917.49it/s]


Phase 6: Creating channel belt and boundaries
Phase 7: Create UTM geodataframe


...

100%|██████████| 1/1 [00:00<00:00, 11.56it/s]

Phase 12: Get bank coordinates for main channel



100%|██████████| 28512/28512 [00:06<00:00, 4096.92it/s] 
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 12.35it/s]

✓ Successfully processed ndwi_mosaic.tif
  - Directed graph: 2 nodes, 1 edges
  - Rook graph: 2 polygons
  - Primal graph: 2 nodes


True

In [55]:
river.summary()


=== River Summary: ndwi_mosaic.tif ===
Processed: True
Successful: True
Start point: (753443.4, 8598434.2)
End point: (743089.7, 8648110.2)
Directed graph: 2 nodes, 1 edges
Rook graph: 2 polygons
Primal graph: 2 nodes
MNDWI shape: (17013, 7701)
Main path: 1 edges
Channel width: 246.7 ± 215.0 m
Channel length: 97553.2 m


### Plot width data for main channel

In [44]:
s, w = river.get_channel_widths()
plt.figure(figsize=(12, 4))
plt.plot(s, w) 
plt.xlabel('along-channel distance (m)')
plt.ylabel('channel width (m)');

### Plot meander wavelength and channel width data

In [45]:
fig, ax = river.plot_overview()
df, curv, s, loc_zero_curv, xsmooth, ysmooth = river.analyze_wavelength_and_width(ax=ax, 
                        delta_s=5.0, smoothing_factor=1e6, min_sinuosity=1.1, dx=3)
df.head()

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 64.47it/s]


,wavelengths (m),sinuosities,mean widths (m),std. dev. of widths (m),along-channel distance (km)
0,1702.798992,1.260533,117.760619,5.325301,3.320376
1,1827.987198,3.536084,110.323100,8.100274,5.472576
2,3849.473068,1.380029,103.869335,5.488447,8.420764
3,2375.920113,1.172625,113.030951,6.276457,11.686329
4,3174.248635,1.334377,107.433117,6.523407,13.435510


## Lena Delta, Russia


Link to data file: https://utexas.box.com/s/4b87y3og9o40zti0cytye0ppecepncy8

In [56]:
# it takes about 45 minutes to run this!

river = rb.River(
    dirname = "data/Lena/",
    fname = 'lena_1_mndwi_clipped.tif',
    start_x = 437535.424,
    start_y = 7970396.089,
    end_x = 522971.474,
    end_y = 8092767.374,
    file_type='water_index'
)

river.map_river_banks(
    ch_belt_smooth_factor=1e8,
    ch_belt_half_width=100000,
    radius = 1000,
    remove_smaller_components=True,
    small_hole_threshold=64,
    solidity_filter=False,
    plot_D_primal=True,
    flip_outlier_edges=True,
    mndwi_threshold=0.0,
    check_edges=False,
    filter_contours=True
)

Processing river: lena_1_mndwi_clipped.tif
Start point: (437535.424, 7970396.089)
End point: (522971.474, 8092767.374)
Phase 1: Initial setup and skeletonization
running skeletonization
Phase 2: Finding main path with fallbacks
Phase 3: Processing graph and extending edges


100%|██████████| 185/185 [01:49<00:00,  1.70it/s]


Phase 4: Extracting and smoothing main path
Phase 5: Polygonizing centerline network


100%|██████████| 7052/7052 [04:37<00:00, 25.39it/s] 


Phase 6: Creating channel belt and boundaries
Phase 7: Create UTM geodataframe


...

100%|██████████| 3607/3607 [00:45<00:00, 80.12it/s]


Phase 12: Get bank coordinates for main channel


100%|██████████| 3632/3632 [00:01<00:00, 1971.73it/s]

✓ Successfully processed lena_1_mndwi_clipped.tif
  - Directed graph: 2379 nodes, 3632 edges
  - Rook graph: 1212 polygons
  - Primal graph: 2379 nodes


True

In [57]:
river.summary()


=== River Summary: lena_1_mndwi_clipped.tif ===
Processed: True
Successful: True
Start point: (437535.4, 7970396.1)
End point: (522971.5, 8092767.4)
Directed graph: 2379 nodes, 3632 edges
Rook graph: 1212 polygons
Primal graph: 2379 nodes
MNDWI shape: (6569, 4600)
Main path: 58 edges
Channel width: 3976.2 ± 2233.4 m
Channel length: 190436.4 m


#### Create polyline that is used to define the coastline and channel termini; use it to get channel mouth polygon

In [58]:
from shapely.geometry import Polygon
# these points can be obtained by using 'ginput':
points = [(494904.8, 7970881.4),
 (502959.4, 7975311.4),
 (502355.3, 7987192.0),
 (502153.9, 7991017.9),
 (504368.9, 7991017.9),
 (509000.3, 7983970.1),
 (515444.0, 7984574.2),
 (520880.9, 7997663.0),
 (523297.2, 8022229.5),
 (523095.9, 8045185.1),
 (522693.2, 8076396.7),
 (521686.3, 8101366.0),
 (513229.0, 8119488.9),
 (501549.8, 8135195.3),
 (488461.1, 8148888.2),
 (471949.1, 8159761.9),
 (453423.6, 8163587.8),
 (438925.3, 8163185.1)]

x_utm, y_utm, ch_map = rb.get_channel_mouth_polygon(river._mndwi, river.dataset, points)
fig, ax = river.plot_mndwi()
ax.plot(x_utm, y_utm, 'g', linewidth=3)
ch_poly = rb.create_channel_nw_polygon(river._G_rook, buffer=10, 
                ch_mouth_poly=Polygon(np.vstack((x_utm, y_utm)).T), 
                dataset=river.dataset)


100%|██████████| 665/665 [00:01<00:00, 375.97it/s] 


#### Recreate directed graph

In [59]:
D_primal, sources, sinks = rb.create_directed_multigraph(river._G_primal,
        river._G_rook, river._xs, river._ys, 1, 108, flip_outlier_edges=True, check_edges=False,
        x_utm=x_utm, y_utm=y_utm)
fig, ax = river.plot_mndwi()
rb.plot_graph_w_colors(D_primal, ax)

12206 7
10933 1890


100%|██████████| 3607/3607 [00:29<00:00, 121.19it/s]


Removed 141 nodes inside the polygon
Added 136 new nodes at edge intersections
Created 136 new boundary nodes
Total sink nodes in truncated graph: 140
Total source nodes in truncated graph: 17


100%|██████████| 3470/3470 [00:01<00:00, 2000.70it/s]
